✅: 미해결 수정사항 || 질문

In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import cv2
from skimage.metrics import structural_similarity
import re #정규표현식
from PIL import Image
from selenium.webdriver.common.by import By

import os
import natsort # 파일명 숫자 정렬(1,2,3, ...)용

import requests

def download(url, file_name):
    with open(file_name, "wb") as file:   # open in binary mode
        response = requests.get(url)       # get request
        file.write(response.content)      # write to fill

In [ ]:
# 드라이버 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(1) 
next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
page_num = 1

# 회사 클릭
company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(i)).get_attribute('innerHTML') # 회사명 저장
ESG = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(i)).click() # i번째 회사 클릭
time.sleep(0.3)

# 보고서 프레임으로 이동
driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
time.sleep(0.3)
driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
time.sleep(0.3)
driver.switch_to.frame("docViewFrm")

# 보고서 링크 클릭
time.sleep(0.3)
Report = driver.find_element(By.CSS_SELECTOR, "body > div > div > a") # 보고서 클릭
Report.click()
driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
URL = driver.current_url # 현재 링크 저장

# 다운로드
download(URL, "ESG_Reports_File/{}.pdf".format(company)) # 회사명으로 파일 저장
driver.close()
driver.switch_to.window(driver.window_handles[-1])
driver.close()


In [ ]:
# 크롤러 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(2) 
page_num = 1
error_count = 0
res = os.listdir("ESG_Reports_File") # 저장 경로
eng_report_list = []

while page_num <= 13:
    print(page_num, "번째 페이지 탐색중 ...")
    for i in range(1, 11):
        # 회사 탐색        
        company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(i)).get_attribute('innerHTML') # 회사명 저장
        print(i, '번째 회사 탐색 : ', company, end=" |")

        for j in range(len(res)):
            if res[j][:-4] == company:
                same = 1
                break
        if same == 1:
            print(company, "는 이미 존재하는 보고서입니다.")
            same = 0
            continue

        # 회사 클릭
        driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(i)).click() # i번째 회사 클릭
        time.sleep(0.6)

        # 보고서 프레임으로 이동
        try:
            driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
            driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
            time.sleep(0.9)
            driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
            time.sleep(0.6)
            driver.switch_to.frame("docViewFrm")
        except:
            if error_count < 3:
                print("탐색 중 오류가 발생해 다시 시도합니다. ", error_count, "회")
                driver.switch_to.window(driver.window_handles[-1])
                driver.close()
                i -= 1
                driver.switch_to.window(driver.window_handles[-1])
                error_count += 1
                continue
        
        # 보고서 링크 클릭
        time.sleep(0.6)

        try: # 영문임을 가정하고 실행
            Report_kor = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(4)") # 보고서 클릭
            Report_eng = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(5)")
            print(company + " 영문 PDF 탐지")
            eng_report_list.append(company)
            
            Report_kor.click()
            Report_eng.click()
            time.sleep(1)

            driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
            URL_eng = driver.current_url # 현재 링크 저장
            driver.close() # 목적 완수 후 보고서 페이지 닫기
            time.sleep(0.2)

            driver.switch_to.window(driver.window_handles[-1])
            URL_kor = driver.current_url # 현재 링크 저장
            driver.close() # 목적 완수 후 보고서 페이지 닫기
            time.sleep(0.2)

            driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
            driver.close() # 공시 페이지 닫기
            time.sleep(0.2)

            print(company + " 영문, 국문 보고서 다운로드")
            download(URL_kor, "ESG_Reports_File/{}_1.pdf".format(company)) # 회사명으로 파일 저장
            download(URL_eng, "ESG_Reports_File/{}_2.pdf".format(company)) # 회사명으로 파일 저장

        except: # 국문 보고서 시도하기
            Report = driver.find_element(By.CSS_SELECTOR, "body > div > div > a").click() # 보고서 클릭
            driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
            URL = driver.current_url # 현재 링크 저장
            
            # 목적 완수 후 보고서 페이지 닫기
            driver.close()
            time.sleep(0.3)
            driver.switch_to.window(driver.window_handles[-1])
            
            driver.close() # 공시 페이지 닫기
            time.sleep(0.2)

            print(company + " 국문 보고서 다운로드")
            download(URL, "ESG_Reports_File/{}.pdf".format(company)) # 회사명으로 파일 저장

        driver.switch_to.window(driver.window_handles[-1]) # 다시 KRX 페이지로 전환
        
    driver.find_element(By.CSS_SELECTOR, "ul > li.next > a").click() # 다음 페이지로 이동하기
    time.sleep(1.5)
    page_num += 1

In [23]:
# 크롤러 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(2) 
page_num = 1
error_count = 0
res = os.listdir("ESG_ENG_Reports_File") # 저장 경로
eng_report_list = []

while page_num <= 13:
    print(page_num, "번째 페이지 탐색중 ...")
    for i in range(1, 11):
        # 회사 탐색        
        company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(i)).get_attribute('innerHTML') # 회사명 저장
        print(i, '번째 회사 탐색 : ', company, end=" |")

        for j in range(len(res)):
            if res[j][:-4] == company:
                same = 1
                break
        if same == 1:
            print(company, "는 이미 존재하는 보고서입니다.")
            same = 0
            continue

        # 회사 클릭
        driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(i)).click() # i번째 회사 클릭
        time.sleep(0.6)

        # 보고서 프레임으로 이동
        try:
            driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
            driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
            time.sleep(0.9)
            driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
            time.sleep(0.6)
            driver.switch_to.frame("docViewFrm")
        except:
            if error_count < 3:
                print("탐색 중 오류가 발생해 다시 시도합니다. ", error_count, "회")
                driver.switch_to.window(driver.window_handles[-1])
                driver.close()
                i -= 1
                driver.switch_to.window(driver.window_handles[-1])
                error_count += 1
                continue
        
        # 보고서 링크 클릭
        time.sleep(0.6)

        try: # 영문임을 가정하고 실행
            Report_kor = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(4)") # 보고서 클릭
            Report_eng = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(5)")
            print(company + " 영문 PDF 탐지")
            eng_report_list.append(company)
            
            Report_kor.click()
            Report_eng.click()
            time.sleep(1)

            driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
            URL_eng = driver.current_url # 현재 링크 저장
            driver.close() # 목적 완수 후 보고서 페이지 닫기
            time.sleep(0.2)

            driver.switch_to.window(driver.window_handles[-1])
            URL_kor = driver.current_url # 현재 링크 저장
            driver.close() # 목적 완수 후 보고서 페이지 닫기
            time.sleep(0.2)

            driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
            driver.close() # 공시 페이지 닫기
            time.sleep(0.2)

            print(company + " 영문, 국문 보고서 다운로드")
            download(URL_kor, "ESG_Reports_File/{}_1.pdf".format(company)) # 회사명으로 파일 저장
            download(URL_eng, "ESG_Reports_File/{}_2.pdf".format(company)) # 회사명으로 파일 저장

        except: # 국문 보고서 시도하기
            print(company, "는 국문 보고서만 존재합니다. 패스합니다.")
            driver.close() # 공시 페이지 닫기
            time.sleep(0.2)
            pass

        driver.switch_to.window(driver.window_handles[-1]) # 다시 KRX 페이지로 전환
        
    driver.find_element(By.CSS_SELECTOR, "ul > li.next > a").click() # 다음 페이지로 이동하기
    time.sleep(1.5)
    page_num += 1

1 번째 페이지 탐색중 ...
1 번째 회사 탐색 :  한국가스공사 |한국가스공사 는 국문 보고서만 존재합니다. 패스합니다.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"tbody > tr:nth-child(2) > td:nth-child(1)"}
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0064DF13+2219795]
	Ordinal0 [0x005E2841+1779777]
	Ordinal0 [0x004F423D+803389]
	Ordinal0 [0x00523025+995365]
	Ordinal0 [0x005231EB+995819]
	Ordinal0 [0x00550F52+1183570]
	Ordinal0 [0x0053E844+1108036]
	Ordinal0 [0x0054F192+1175954]
	Ordinal0 [0x0053E616+1107478]
	Ordinal0 [0x00517F89+950153]
	Ordinal0 [0x00518F56+954198]
	GetHandleVerifier [0x00942CB2+3040210]
	GetHandleVerifier [0x00932BB4+2974420]
	GetHandleVerifier [0x006E6A0A+565546]
	GetHandleVerifier [0x006E5680+560544]
	Ordinal0 [0x005E9A5C+1808988]
	Ordinal0 [0x005EE3A8+1827752]
	Ordinal0 [0x005EE495+1827989]
	Ordinal0 [0x005F80A4+1867940]
	BaseThreadInitThunk [0x7533FA29+25]
	RtlGetAppContainerNamedObjectPath [0x771C7B5E+286]
	RtlGetAppContainerNamedObjectPath [0x771C7B2E+238]


In [ ]:
try:
    #image.click()
    driver.execute_script("arguments[0].click();", image) 
    time.sleep(0.7) 
    image_url = driver.find_element_by_xpath("/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img").get_attribute("src")

except:
    pass

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
count = 1
images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
createFolder(save_path)

driver.quit() # 창 닫기

# 이미지 경로 정렬
dir = os.getcwd()
file_path = os.path.join(dir+'/Style_Images/'+search_keyword)
file_names = os.listdir(file_path)
file_names = natsort.natsorted(file_names) # 숫자 순으로 정렬

### 중복 이미지 제거

In [12]:
# 중복 이미지 제거할 이미지 리스트 정의
img_list = []

for f in file_names:
    if '.jpg' in f:
        img_list.append(f)
        
img_size = list(map(lambda x: os.path.getsize(file_path + '/' + x), img_list))

# Dataframe 정의
img_df = pd.DataFrame({'filename_raw':img_list, 'size':img_size})
print('총 이미지 수:', len(img_df))
img_df.head()

# 파일명 숫자 제거
com = re.compile(' \d')
img_df['filename'] = list(map(lambda x: com.sub('', x), img_list))

# 이미지명 & 이미지 크기 확인
filename_df= pd.DataFrame({'filename':img_df['filename'].value_counts().index, 'filename_counts':img_df['filename'].value_counts().values})   
filesize_df = pd.DataFrame({'size':img_df['size'].value_counts().index, 'size_counts':img_df['size'].value_counts().values})   

img_df = pd.merge(img_df, filename_df, how = 'left', on = 'filename')
img_df = pd.merge(img_df, filesize_df, how = 'left', on = 'size')

img_df.head()

### 1) 동일 파일명 || 동일 사이즈 이미지 제거

In [17]:
# 기존 dataframe에서 동일 파일명 이미지 제거
img_df2 = img_df.sort_values(['filename_raw'], ascending = True).drop_duplicates(['filename'], keep = 'first')

print('삭제된 이미지 수:',len(img_df)-len(img_df2))
print('남은 이미지 수:', len(img_df2)) 

filename_df2 = pd.DataFrame({'filename':img_df2['filename'].value_counts().index, 'filename_counts2':img_df2['filename'].value_counts().values})   
filesize_df2 = pd.DataFrame({'size':img_df2['size'].value_counts().index, 'size_counts2':img_df2['size'].value_counts().values})   

img_df2 = pd.merge(img_df2, filename_df2, how = 'left', on = 'filename')
img_df2 = pd.merge(img_df2, filesize_df2, how = 'left', on = 'size')

img_df2.head()

In [20]:
print('사이즈 중복된 이미지 수:', len(img_df2[img_df2['size_counts2']!=1]))
print('중복 사이즈 수:', len(filesize_df2[filesize_df2['size_counts2']>1]))

사이즈 중복된 이미지 수: 8
중복 사이즈 수: 4


In [21]:
# 삭제될 이미지 리스트
img_delete_list = []

for i in range(len(filesize_df2)):
    
    # 중복된 크기(size)가 있는 경우
    if filesize_df2['size_counts2'][i] == 2:
        
        # 그 크기에 해당하는 이미지 불러옴
        temp = img_df2[img_df2['size']==filesize_df2['size'][i]].reset_index(drop = True).sort_values(['filename'])
        
        imageA = cv2.imread(file_path+'/'+temp['filename_raw'][0])
        imageB = cv2.imread(file_path+'/'+temp['filename_raw'][1])
        
        grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        
        # if) 이미지 구조 same -> 이미지 비교
        if len(grayA)==len(grayB):
            (score, diff) = structural_similarity(grayA, grayB, full=True)
            
            # if) 차이 無 -> 둘중 한 장 img_delete_list에 추가
            if score == 1:
                img_delete_list.append(temp['filename_raw'][1])
            
            # if) 구조 same but 차이 有 -> 직접 확인     
            else:
                print('이미지 중복 여부 확인 필요: ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score: {score})')
                
img_delete_list

['sample37.jpg', 'sample44.jpg', 'sample50.jpg', 'sample42.jpg']

In [22]:
# 중복 제거된 것들 img_delete_list에 넣어주기
img_delete_list = img_delete_list + list(img_df[~img_df['filename_raw'].isin(img_df2['filename_raw'])]['filename_raw'])
print('삭제할 이미지 수:', len(img_delete_list))

삭제할 이미지 수: 4


In [23]:
# img_remained_list: 처음 dataframe에서 img_delete_list 제외
img_remained_list = list(img_df[~img_df['filename_raw'].isin(img_delete_list)]['filename_raw'])
print('남은 이미지 수:', len(img_remained_list))

남은 이미지 수: 141


In [24]:
# 중복 이미지 제거
for img in img_delete_list:
    route = os.path.join(file_path+'/'+img)
    os.remove(route)

In [25]:
# 이미지 remove 후 파일명 리스트 재정의
dir = os.getcwd()
file_path = os.path.join(dir+'/Style_Images/'+search_keyword)
file_names = os.listdir(file_path)
file_names = natsort.natsorted(file_names)

### 2) 다른 사이즈 && 동일 이미지 제거 - ✅ 에러 해결 中

### 3) 256*256 미만 사이즈 이미지 제거

In [26]:
img_delete_list3 = []
for name in file_names:
    route = os.path.join(file_path+'/'+name)
    img = cv2.imread(route)
    height, width, color = img.shape
    if width < 256 and height < 256:
        img_delete_list3.append(name)
img_delete_list3

['sample102.jpg']

In [27]:
# 크기 작은 이미지 제거
for small_img in img_delete_list3:
    img = os.path.join(file_path+'/'+small_img)
    os.remove(img)

In [28]:
# 이미지 remove 후 파일명 리스트 재정의
dir = os.getcwd()
file_path = os.path.join(dir+'/Style_Images/'+search_keyword)
file_names = os.listdir(file_path)
file_names = natsort.natsorted(file_names)

In [29]:
# 남은 이미지 다시 넘버링하여 저장
count = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = search_keyword + str(count) + '.jpg' 
    dst = os.path.join(file_path, dst)
    try:
        os.rename(src, dst)
    except FileExistsError:
        print("File already exists: Removing prior file")
        os.remove(dst)
        os.rename(src, dst)
    count += 1

In [30]:
# 변경된 파일명 확인
res = os.listdir(file_path)
res = natsort.natsorted(res)
res

['sample1.jpg',
 'sample2.jpg',
 'sample3.jpg',
 'sample4.jpg',
 'sample5.jpg',
 'sample6.jpg',
 'sample7.jpg',
 'sample8.jpg',
 'sample9.jpg',
 'sample10.jpg',
 'sample11.jpg',
 'sample12.jpg',
 'sample13.jpg',
 'sample14.jpg',
 'sample15.jpg',
 'sample16.jpg',
 'sample17.jpg',
 'sample18.jpg',
 'sample19.jpg',
 'sample20.jpg',
 'sample21.jpg',
 'sample22.jpg',
 'sample23.jpg',
 'sample24.jpg',
 'sample25.jpg',
 'sample26.jpg',
 'sample27.jpg',
 'sample28.jpg',
 'sample29.jpg',
 'sample30.jpg',
 'sample31.jpg',
 'sample32.jpg',
 'sample33.jpg',
 'sample34.jpg',
 'sample35.jpg',
 'sample36.jpg',
 'sample37.jpg',
 'sample38.jpg',
 'sample39.jpg',
 'sample40.jpg',
 'sample41.jpg',
 'sample42.jpg',
 'sample43.jpg',
 'sample44.jpg',
 'sample45.jpg',
 'sample46.jpg',
 'sample47.jpg',
 'sample48.jpg',
 'sample49.jpg',
 'sample50.jpg',
 'sample51.jpg',
 'sample52.jpg',
 'sample53.jpg',
 'sample54.jpg',
 'sample55.jpg',
 'sample56.jpg',
 'sample57.jpg',
 'sample58.jpg',
 'sample59.jpg',
 'samp